In [1]:
import pandas as pd
import os
import glob

In [2]:
# Get the current working directory
current_directory = os.getcwd() + "\\Test_data\\historical_csv_data\\England"
print(current_directory)

# Define the pattern to match all CSV files in the current directory
csv_files = glob.glob(os.path.join(current_directory, "*.csv"))

# Create an empty list to store individual DataFrames
df_list = []

# Loop through each CSV file, read it into a DataFrame, and append to the list
for file in csv_files:
    df = pd.read_csv(file)
    df_list.append(df)
df = pd.concat(df_list)
df.sort_values(by="Date")
df

c:\Dev\Betting project\Code\betting_project-1\Test_data\historical_csv_data\England


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA
0,E0,05/08/2022,20:00,Crystal Palace,Arsenal,0,2,A,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,06/08/2022,12:30,Fulham,Liverpool,2,2,D,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,06/08/2022,15:00,Bournemouth,Aston Villa,2,0,H,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,06/08/2022,15:00,Leeds,Wolves,2,1,H,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,06/08/2022,15:00,Newcastle,Nott'm Forest,2,0,H,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,SP1,18/05/2019,NaN,Levante,Ath Madrid,2,2,D,2.0,0.0,...,1.71,1.65,2.31,2.23,18.0,0.5,1.91,1.87,2.04,1.99
376,SP1,18/05/2019,NaN,Sevilla,Ath Bilbao,2,0,H,1.0,0.0,...,1.81,1.74,2.16,2.08,19.0,-1.0,2.70,2.60,1.60,1.53
377,SP1,18/05/2019,NaN,Valladolid,Valencia,0,2,A,0.0,1.0,...,1.69,1.63,2.33,2.25,20.0,1.5,1.78,1.74,2.20,2.13
378,SP1,19/05/2019,NaN,Eibar,Barcelona,2,2,D,2.0,2.0,...,1.50,1.46,2.85,2.65,19.0,0.5,2.03,1.98,1.92,1.88


In [3]:
from datetime import datetime

from Models.H2hEvent import H2hBookmaker, H2hEvent, H2hOutcome

BOOKMAKERS = {
    "1XB": "1XBet",
    "B365": "Bet365",
    "BF": "Betfair (Sportsbook)",
    "BFE": "Betfair Exchange",
    "BMG": "Betfred",
    "BFD": "Betfred",
    "BV": "Betvictor",
    "BS": "Blue Square",
    "BW": "Bet and Win",
    "CL": "Coral",
    "GB": "GameBookers",
    "IW": "Interwetten",
    "LB": "Ladbrokes",
    "PS": "Pinnacle",
    "P": "Pinnacle",
    "SO": "Sporting Odds",
    "SB": "Sporting Bet",
    "SJ": "Stan James",
    "SY": "Stanley bet",
    "VC": "VC Bet",
    "WH": "William Hill"
}

allEvents = []

for index, event in df.iterrows():
    id = event["Date"] + event["HomeTeam"] + event["AwayTeam"]
    sport_key = id

    date_string = f"{event["Date"]} 00:00"
    format_string = "%d/%m/%Y %H:%M"
    parsed_dt = datetime.strptime(date_string, format_string)
    commence_time = parsed_dt

    home_team = event["HomeTeam"]
    away_team = event["AwayTeam"]

    foundBookmakers = []
    for key, value in BOOKMAKERS.items():
        home_key = f"{key}H"
        away_key = f"{key}A"
        draw_key = f"{key}D"

        if(any(item not in event.keys() for item in [home_key, away_key, draw_key])):
            continue

        foundBookmakers.append(
            H2hBookmaker(
                title=value,
                home_team_back_odds=event[home_key],
                away_team_back_odds=event[away_key],
                draw_back_odds=event[draw_key]
            )
        )

    outcome = H2hOutcome[event["FTR"]]
    allEvents.append(
        H2hEvent(
            id=id,
            sport_key=sport_key,
            home_team=home_team,
            away_team=away_team,
            commence_time=commence_time,
            bookmakers=foundBookmakers,
            outcome=outcome,
        )
    )

pd.DataFrame(allEvents)

,id,sport_key,commence_time,home_team,away_team,bookmakers,outcome
0,05/08/2022Crystal PalaceArsenal,05/08/2022Crystal PalaceArsenal,2022-08-05,Crystal Palace,Arsenal,"[{'title': '1XBet', 'home_team_back_odds': nan...",H2hOutcome.A
1,06/08/2022FulhamLiverpool,06/08/2022FulhamLiverpool,2022-08-06,Fulham,Liverpool,"[{'title': '1XBet', 'home_team_back_odds': nan...",H2hOutcome.D
2,06/08/2022BournemouthAston Villa,06/08/2022BournemouthAston Villa,2022-08-06,Bournemouth,Aston Villa,"[{'title': '1XBet', 'home_team_back_odds': nan...",H2hOutcome.H
3,06/08/2022LeedsWolves,06/08/2022LeedsWolves,2022-08-06,Leeds,Wolves,"[{'title': '1XBet', 'home_team_back_odds': nan...",H2hOutcome.H
4,06/08/2022NewcastleNott'm Forest,06/08/2022NewcastleNott'm Forest,2022-08-06,Newcastle,Nott'm Forest,"[{'title': '1XBet', 'home_team_back_odds': nan...",H2hOutcome.H
...,...,...,...,...,...,...,...
18001,18/05/2019LevanteAth Madrid,18/05/2019LevanteAth Madrid,2019-05-18,Levante,Ath Madrid,"[{'title': '1XBet', 'home_team_back_odds': nan...",H2hOutcome.D
18002,18/05/2019SevillaAth Bilbao,18/05/2019SevillaAth Bilbao,2019-05-18,Sevilla,Ath Bilbao,"[{'title': '1XBet', 'home_team_back_odds': nan...",H2hOutcome.H
18003,18/05/2019ValladolidValencia,18/05/2019ValladolidValencia,2019-05-18,Valladolid,Valencia,"[{'title': '1XBet', 'home_team_back_odds': nan...",H2hOutcome.A
18004,19/05/2019EibarBarcelona,19/05/2019EibarBarcelona,2019-05-19,Eibar,Barcelona,"[{'title': '1XBet', 'home_team_back_odds': nan...",H2hOutcome.D


In [ ]:
import Betting_Engines.H2hEventAboveMeanOddsBettingEngine as BettingEngine
import importlib
import sys

importlib.reload(BettingEngine)

bettingEngine = BettingEngine.H2hEventAboveMeanOddsBettingEngine(
    alpha=0.059,
    commision=0.00,
    betAmount=1000,
    betOddsUpperLimit=2.25
)

foundBets = pd.DataFrame(bettingEngine.analyseAndFindBets(allEvents))
# .query("bookmaker=='Betfair Exchange'")


c:\Dev\Betting project\Code\betting_project-1\Betting_Engines\H2hEventAboveMeanOddsBettingEngine.py:22: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  highest_bookmaker_index = bookmakers_df[f"{outcome}_back_odds"].idxmax()
c:\Dev\Betting project\Code\betting_project-1\Betting_Engines\H2hEventAboveMeanOddsBettingEngine.py:22: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  highest_bookmaker_index = bookmakers_df[f"{outcome}_back_odds"].idxmax()
c:\Dev\Betting project\Code\betting_project-1\Betting_Engines\H2hEventAboveMeanOddsBettingEngine.py:22: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  highest_bookmaker_index = bookmakers_df[f"{outcome}_back_odds"].idxmax(

In [5]:

foundBets["bookmaker_odds_as_probability"] = 1/foundBets.bookmaker_average_odds
for index, bet in foundBets.iterrows():
    foundBets.at[index, "is_correct_guess"] = bet["guessed_outcome"] == bet["outcome"]

for index, bet in foundBets.iterrows():
    foundBets.at[index, "profit"] = (
        bet["betAmount"] * (bet["odds"] - 1) * 0.96
        if bet["is_correct_guess"]
        else -1 * bet["betAmount"]
    )

profit =foundBets["profit"].sum()
spent = foundBets["betAmount"].sum()


print(f"number of bets: {len(foundBets)}")
print(f"profit: {profit}")
print(f"spent: {spent}")

roi = profit/spent
print(f"return: {roi}")

print("\n accuracy analysis")
foundBets["odds_as_probability"] = 1/foundBets.odds
meanOddsProbability = foundBets.odds_as_probability.mean()
oddsStd = foundBets.odds_as_probability.std()
percentagePredictedCorrectly = foundBets.is_correct_guess.sum() / len(foundBets)

print(f"number predicted correct: {percentagePredictedCorrectly}")
print(f"mean odds probability: {meanOddsProbability}")
print(f"stdev: {oddsStd}")
z_score = (percentagePredictedCorrectly - meanOddsProbability) / oddsStd
print(f"z_score: {z_score}")

print("\n returns analysis")
meanBookmakerExpectedReturn = 0
profitStd = foundBets.profit.std()

print(f"mean profit: {foundBets.profit.mean()}, roi: {roi}")
print(f"mean bookmaker return: {meanBookmakerExpectedReturn}")
print(f"stdev: {profitStd}")
z_score = (foundBets.profit.mean() - meanBookmakerExpectedReturn) / profitStd
print(f"z_score: {z_score}")

print(foundBets.to_string())

number of bets: 28
profit: 9633.599999999999
spent: 28000
return: 0.3440571428571428

 accuracy analysis
number predicted correct: 0.6785714285714286
mean odds probability: 0.5140909352113843
stdev: 0.09272356554774959
z_score: 1.773880160759589

 returns analysis
mean profit: 344.0571428571428, roi: 0.3440571428571428
mean bookmaker return: 0
stdev: 976.1075818927807
z_score: 0.3524787116087941
                                    id         bookmaker      home_team      away_team commence_time  odds                           sport_key guessed_outcome  betAmount  bookmaker_average_odds       outcome  bookmaker_odds_as_probability is_correct_guess  profit  odds_as_probability
0             03/11/2018ValenciaGirona       Interwetten       Valencia         Girona    2018-11-03  1.65            03/11/2018ValenciaGirona    H2hOutcome.H       1000                   1.500  H2hOutcome.A                       0.666667            False -1000.0             0.606061
1          24/10/2021AjaxPSV Ei